In [9]:
import pandas as pd
df = pd.read_excel('data/Ecoli/genes_and_promoters.xlsx', sheet_name='all_clear_v2')
df

start, end = 190, 9191
genes_of_interval = df[(df['start'] >= start) & \
                       (df['end'] <= end) & \
                       (df['class'] == 'gene')]
genes_of_interval['gene_or_promoter'].tolist()

[u'b0001',
 u'b0002',
 u'b0003',
 u'b0004',
 u'b0005',
 u'b0006',
 u'b0007',
 u'b0008']